In [2]:
import pandas as pd
import os
import numpy as np

In [3]:
def change_result(a):
    if a != 'PASS':
        a = 'FAIL'
    return a

In [4]:
files = os.listdir('./Log')

In [5]:
barcode_df = pd.read_excel('./barcode.xlsx') 

In [6]:
barcode_df['Result'] = barcode_df['Result'].apply(change_result)

In [7]:
barcode_fail = barcode_df.iloc[:,0].to_list()

In [8]:
item_take = pd.read_excel('./item.xlsx',header = None)

In [9]:
item_take = item_take.iloc[:,0].to_list()

In [10]:
sfr_df = pd.DataFrame()

In [11]:
for file in files:
    file_1 = file
    print(file_1)
    sfr_df_temp = pd.read_csv('./Log/'+file_1,error_bad_lines = False)
    sfr_df_temp = sfr_df_temp.loc[:,item_take]
    sfr_df_temp = sfr_df_temp[sfr_df_temp['barcode'].isin(barcode_fail)]
    sfr_df = sfr_df.append(sfr_df_temp, ignore_index=True)

8g4e3fovptr6mr2j7g96_AIchartMacroUpCircular.CSV
aae26aanfs8emx9a21uf_AIchartMacroUpCircular.CSV


In [12]:
sfr_df = sfr_df.merge(barcode_df,how = 'inner',left_on = 'barcode',right_on='Module ID').drop('Module ID',axis = 1)

In [13]:
#sfr_df.to_excel('./Result_merge.xlsx',index= False)

In [14]:
sfr_df['barcode'] = sfr_df['Machine']
sfr_df = sfr_df.drop('Machine',axis = 1)
sfr_df.rename(columns={'barcode':'Machine'},inplace =  True)

In [15]:
sfr_df.head()

,Machine,station,site,sfr_40cm_AF_Code_Peak,sfr_40cm_ny4_ROI_index,sfr_40cm_Ny4_cen_diff,sfr_40cm_Ny4_cen_min_tan,sfr_40cm_Ny4_30F_min_tan,sfr_40cm_Ny4_60F_min_tan,sfr_40cm_Ny4_75F_min_tan,...,sfr_40cm_Ny4_TB_45F_Delta,sfr_40cm_Ny4_LR_45F_Delta,sfr_40cm_Ny4_TB_70F_Delta,sfr_40cm_Ny4_LR_70F_Delta,sfr_40cm_Ny4_TB_70F_Delta2,sfr_40cm_Ny4_LR_70F_Delta2,sfr_40cm_Ny4_Max_Inner_Delta,sfr_40cm_Ny4_Max_Mid_Delta,sfr_40cm_Ny4_Max_Edge_Delta,Result
0,5118,VCMCT8031T02,84,296,149,0.0124999880,0.6237630844,0.4883560538,0.2858482003,0.2948465347,...,0.0186776121,0.0384443402,0.0117540955,0.0558722689,0.0172812417,0.0546278432,0.0668344497,0.1169635653,0.0586217194,PASS
1,5118,VCMCT8031T02,76,310,232,0.0149661302,0.6168941259,0.4444400072,0.2468225508,0.2396168410,...,0.0180660635,0.0401426007,0.0241163671,0.0644936710,0.0287704169,0.0634578466,0.0808145701,0.1269736438,0.0477399080,PASS
2,5118,VCMCT8031T02,71,300,0,0.0161617994,0.6245274543,0.4973722696,0.3033604025,0.2946655750,...,0.0256367027,0.0323909819,0.0138742104,0.0532785803,0.0198920145,0.0478283688,0.0628958046,0.1015708148,0.0602397918,PASS
3,5118,VCMCT8031T02,48,294,232,0.0207970142,0.5913621187,0.3772543668,0.2338746786,0.1919809281,...,-0.0080601723,0.0304125497,0.0305701997,0.0363700296,0.0336762741,0.0377082154,0.1019826382,0.0821736156,0.1219132989,PASS
4,5118,VCMCT8031T02,79,280,232,0.0127397179,0.6285438537,0.5135080814,0.3387429118,0.2767354547,...,0.0081016023,0.0282781124,0.0335786640,0.0174813568,0.0404629558,0.0175551325,0.0626291632,0.0738602876,0.1074805855,PASS


In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [17]:
X = sfr_df.iloc[:,:-1].values
y = sfr_df.iloc[:, -1].values

In [18]:
ct = ColumnTransformer(transformers=[("encoder",OneHotEncoder(),[0,1,2])],remainder="passthrough")
X = np.array(ct.fit_transform(X))

In [19]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2,random_state= 1)

In [23]:
scaler = StandardScaler()
X_train[:,57:] = scaler.fit_transform(X_train[:,57:])
X_test[:,57:] = scaler.transform(X_test[:,57:])

In [31]:
from catboost import CatBoostClassifier, Pool



cat_model = CatBoostClassifier(

    iterations = 1000, # Thử mẫu 1000 iter

    loss_function='MultiClass',

    bootstrap_type = "Bayesian",

    eval_metric = 'MultiClass',

    leaf_estimation_iterations = 100,

    random_strength = 0.5,

    depth = 7,

    l2_leaf_reg = 5,

    learning_rate=0.25,

    bagging_temperature = 0.5,

    task_type = "CPU",

)



# train

cat_model.fit(X_train,Y_train)



# Thử gia cát zự

y_pred_cat = cat_model.predict(X_test)

# độ ngọc trinh

print("accuracy of the catboost: ",accuracy_score(Y_test,y_pred_cat))



# Confusion matrix

confusion_matrix_LightGBM = confusion_matrix(Y_test,y_pred_cat)

print(confusion_matrix_LightGBM) 

0:	learn: 0.6172456	total: 182ms	remaining: 3m 2s
1:	learn: 0.5610058	total: 324ms	remaining: 2m 41s
2:	learn: 0.5209756	total: 474ms	remaining: 2m 37s
3:	learn: 0.4874761	total: 614ms	remaining: 2m 32s
4:	learn: 0.4591645	total: 769ms	remaining: 2m 33s
5:	learn: 0.4354985	total: 913ms	remaining: 2m 31s
6:	learn: 0.4217365	total: 1.07s	remaining: 2m 32s
7:	learn: 0.4052531	total: 1.22s	remaining: 2m 30s
8:	learn: 0.3858110	total: 1.4s	remaining: 2m 34s
9:	learn: 0.3729936	total: 1.56s	remaining: 2m 34s
10:	learn: 0.3551119	total: 1.72s	remaining: 2m 34s
11:	learn: 0.3454232	total: 1.87s	remaining: 2m 33s
12:	learn: 0.3326249	total: 2.02s	remaining: 2m 33s
13:	learn: 0.3199296	total: 2.19s	remaining: 2m 34s
14:	learn: 0.3083298	total: 2.33s	remaining: 2m 33s
15:	learn: 0.3009353	total: 2.49s	remaining: 2m 33s
16:	learn: 0.2900569	total: 2.65s	remaining: 2m 33s
17:	learn: 0.2778085	total: 2.79s	remaining: 2m 32s
18:	learn: 0.2676311	total: 2.94s	remaining: 2m 31s
19:	learn: 0.2565673	tot

158:	learn: 0.0058469	total: 24.3s	remaining: 2m 8s
159:	learn: 0.0057663	total: 24.5s	remaining: 2m 8s
160:	learn: 0.0056184	total: 24.6s	remaining: 2m 8s
161:	learn: 0.0054625	total: 24.7s	remaining: 2m 8s
162:	learn: 0.0053365	total: 24.9s	remaining: 2m 7s
163:	learn: 0.0052498	total: 25s	remaining: 2m 7s
164:	learn: 0.0051463	total: 25.2s	remaining: 2m 7s
165:	learn: 0.0050332	total: 25.3s	remaining: 2m 7s
166:	learn: 0.0049387	total: 25.5s	remaining: 2m 6s
167:	learn: 0.0048448	total: 25.6s	remaining: 2m 6s
168:	learn: 0.0047875	total: 25.8s	remaining: 2m 6s
169:	learn: 0.0046886	total: 25.9s	remaining: 2m 6s
170:	learn: 0.0046200	total: 26.1s	remaining: 2m 6s
171:	learn: 0.0045587	total: 26.3s	remaining: 2m 6s
172:	learn: 0.0044944	total: 26.4s	remaining: 2m 6s
173:	learn: 0.0044324	total: 26.6s	remaining: 2m 6s
174:	learn: 0.0043479	total: 26.7s	remaining: 2m 5s
175:	learn: 0.0042645	total: 26.9s	remaining: 2m 5s
176:	learn: 0.0042000	total: 27s	remaining: 2m 5s
177:	learn: 0.00

315:	learn: 0.0007616	total: 47.9s	remaining: 1m 43s
316:	learn: 0.0007540	total: 48s	remaining: 1m 43s
317:	learn: 0.0007502	total: 48.2s	remaining: 1m 43s
318:	learn: 0.0007437	total: 48.3s	remaining: 1m 43s
319:	learn: 0.0007360	total: 48.5s	remaining: 1m 43s
320:	learn: 0.0007325	total: 48.7s	remaining: 1m 42s
321:	learn: 0.0007260	total: 48.8s	remaining: 1m 42s
322:	learn: 0.0007210	total: 49s	remaining: 1m 42s
323:	learn: 0.0007144	total: 49.1s	remaining: 1m 42s
324:	learn: 0.0007119	total: 49.3s	remaining: 1m 42s
325:	learn: 0.0007063	total: 49.4s	remaining: 1m 42s
326:	learn: 0.0006995	total: 49.6s	remaining: 1m 42s
327:	learn: 0.0006916	total: 49.7s	remaining: 1m 41s
328:	learn: 0.0006854	total: 49.9s	remaining: 1m 41s
329:	learn: 0.0006799	total: 50s	remaining: 1m 41s
330:	learn: 0.0006775	total: 50.2s	remaining: 1m 41s
331:	learn: 0.0006754	total: 50.3s	remaining: 1m 41s
332:	learn: 0.0006700	total: 50.5s	remaining: 1m 41s
333:	learn: 0.0006620	total: 50.6s	remaining: 1m 40s

472:	learn: 0.0002875	total: 1m 11s	remaining: 1m 19s
473:	learn: 0.0002861	total: 1m 11s	remaining: 1m 19s
474:	learn: 0.0002856	total: 1m 11s	remaining: 1m 18s
475:	learn: 0.0002844	total: 1m 11s	remaining: 1m 18s
476:	learn: 0.0002830	total: 1m 11s	remaining: 1m 18s
477:	learn: 0.0002815	total: 1m 11s	remaining: 1m 18s
478:	learn: 0.0002799	total: 1m 11s	remaining: 1m 18s
479:	learn: 0.0002788	total: 1m 12s	remaining: 1m 18s
480:	learn: 0.0002775	total: 1m 12s	remaining: 1m 17s
481:	learn: 0.0002759	total: 1m 12s	remaining: 1m 17s
482:	learn: 0.0002745	total: 1m 12s	remaining: 1m 17s
483:	learn: 0.0002733	total: 1m 12s	remaining: 1m 17s
484:	learn: 0.0002715	total: 1m 12s	remaining: 1m 17s
485:	learn: 0.0002703	total: 1m 12s	remaining: 1m 17s
486:	learn: 0.0002689	total: 1m 13s	remaining: 1m 16s
487:	learn: 0.0002676	total: 1m 13s	remaining: 1m 16s
488:	learn: 0.0002665	total: 1m 13s	remaining: 1m 16s
489:	learn: 0.0002646	total: 1m 13s	remaining: 1m 16s
490:	learn: 0.0002637	total:

626:	learn: 0.0001619	total: 1m 33s	remaining: 55.6s
627:	learn: 0.0001614	total: 1m 33s	remaining: 55.4s
628:	learn: 0.0001608	total: 1m 33s	remaining: 55.3s
629:	learn: 0.0001602	total: 1m 33s	remaining: 55.1s
630:	learn: 0.0001596	total: 1m 34s	remaining: 55s
631:	learn: 0.0001591	total: 1m 34s	remaining: 54.8s
632:	learn: 0.0001585	total: 1m 34s	remaining: 54.7s
633:	learn: 0.0001577	total: 1m 34s	remaining: 54.6s
634:	learn: 0.0001575	total: 1m 34s	remaining: 54.4s
635:	learn: 0.0001570	total: 1m 34s	remaining: 54.3s
636:	learn: 0.0001567	total: 1m 34s	remaining: 54.1s
637:	learn: 0.0001561	total: 1m 35s	remaining: 54s
638:	learn: 0.0001554	total: 1m 35s	remaining: 53.8s
639:	learn: 0.0001552	total: 1m 35s	remaining: 53.7s
640:	learn: 0.0001549	total: 1m 35s	remaining: 53.5s
641:	learn: 0.0001548	total: 1m 35s	remaining: 53.4s
642:	learn: 0.0001542	total: 1m 35s	remaining: 53.2s
643:	learn: 0.0001537	total: 1m 36s	remaining: 53.1s
644:	learn: 0.0001533	total: 1m 36s	remaining: 52.

783:	learn: 0.0001048	total: 1m 57s	remaining: 32.3s
784:	learn: 0.0001046	total: 1m 57s	remaining: 32.1s
785:	learn: 0.0001043	total: 1m 57s	remaining: 32s
786:	learn: 0.0001040	total: 1m 57s	remaining: 31.8s
787:	learn: 0.0001038	total: 1m 57s	remaining: 31.7s
788:	learn: 0.0001037	total: 1m 57s	remaining: 31.5s
789:	learn: 0.0001035	total: 1m 58s	remaining: 31.4s
790:	learn: 0.0001033	total: 1m 58s	remaining: 31.2s
791:	learn: 0.0001031	total: 1m 58s	remaining: 31.1s
792:	learn: 0.0001029	total: 1m 58s	remaining: 30.9s
793:	learn: 0.0001026	total: 1m 58s	remaining: 30.8s
794:	learn: 0.0001025	total: 1m 58s	remaining: 30.6s
795:	learn: 0.0001022	total: 1m 58s	remaining: 30.5s
796:	learn: 0.0001021	total: 1m 59s	remaining: 30.3s
797:	learn: 0.0001018	total: 1m 59s	remaining: 30.2s
798:	learn: 0.0001015	total: 1m 59s	remaining: 30s
799:	learn: 0.0001013	total: 1m 59s	remaining: 29.9s
800:	learn: 0.0001009	total: 1m 59s	remaining: 29.7s
801:	learn: 0.0001008	total: 1m 59s	remaining: 29.

940:	learn: 0.0000772	total: 2m 21s	remaining: 8.84s
941:	learn: 0.0000771	total: 2m 21s	remaining: 8.69s
942:	learn: 0.0000769	total: 2m 21s	remaining: 8.54s
943:	learn: 0.0000767	total: 2m 21s	remaining: 8.39s
944:	learn: 0.0000765	total: 2m 21s	remaining: 8.24s
945:	learn: 0.0000762	total: 2m 21s	remaining: 8.1s
946:	learn: 0.0000761	total: 2m 22s	remaining: 7.95s
947:	learn: 0.0000760	total: 2m 22s	remaining: 7.8s
948:	learn: 0.0000759	total: 2m 22s	remaining: 7.65s
949:	learn: 0.0000758	total: 2m 22s	remaining: 7.5s
950:	learn: 0.0000757	total: 2m 22s	remaining: 7.35s
951:	learn: 0.0000756	total: 2m 22s	remaining: 7.2s
952:	learn: 0.0000753	total: 2m 22s	remaining: 7.05s
953:	learn: 0.0000751	total: 2m 23s	remaining: 6.9s
954:	learn: 0.0000750	total: 2m 23s	remaining: 6.75s
955:	learn: 0.0000749	total: 2m 23s	remaining: 6.6s
956:	learn: 0.0000748	total: 2m 23s	remaining: 6.45s
957:	learn: 0.0000747	total: 2m 23s	remaining: 6.3s
958:	learn: 0.0000745	total: 2m 23s	remaining: 6.15s
